In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls -l drive/MyDrive/
# unzip cub.zip to data/
!mkdir data/
!unzip drive/MyDrive/cub.zip -d data/
# download pre-trained models
!mkdir models/
!wget https://download.pytorch.org/models/resnet50-0676ba61.pth -O models/resnet50-0676ba61.pth
!wget https://download.pytorch.org/models/resnet101-63fe2227.pth -O models/resnet101-63fe2227.pth
!wget https://download.pytorch.org/models/resnet152-394f9c45.pth -O models/resnet152-394f9c45.pth

In [ ]:
import argparse
import torch

from ret_benchmark.config import cfg
from ret_benchmark.data import build_data
from ret_benchmark.engine.trainer import do_train
from ret_benchmark.losses import build_loss
from ret_benchmark.modeling import build_model
from ret_benchmark.solver import build_lr_scheduler, build_optimizer
from ret_benchmark.utils.logger import setup_logger
from ret_benchmark.utils.checkpoint import Checkpointer

In [ ]:
def train(cfg):
    logger = setup_logger(name='Train', level=cfg.LOGGER.LEVEL)
    logger.info(cfg)
    model = build_model(cfg)
    cfg.MODEL.DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = torch.device(cfg.MODEL.DEVICE)
    model.to(device)

    criterion = build_loss(cfg)

    optimizer = build_optimizer(cfg, model)
    scheduler = build_lr_scheduler(cfg, optimizer)

    train_loader = build_data(cfg, is_train=True)
    val_loader = build_data(cfg, is_train=False)

    logger.info(train_loader.dataset)
    logger.info(val_loader.dataset)

    arguments = dict()
    arguments["iteration"] = 0

    checkpoint_period = cfg.SOLVER.CHECKPOINT_PERIOD
    checkpointer = Checkpointer(model, optimizer, scheduler, cfg.SAVE_DIR)

    do_train(
        cfg,
        model,
        train_loader,
        val_loader,
        optimizer,
        scheduler,
        criterion,
        checkpointer,
        device,
        checkpoint_period,
        arguments,
        logger
    )

In [ ]:
# python tools\main.py --cfg configs\example_resnet50.yaml
cfg.merge_from_file('configs/example_resnet101_margin.yaml')
train(cfg)

In [ ]:
!mkdir drive/MyDrive/resnet101

In [ ]:
!cp output/model_003000.pth drive/MyDrive/resnet101

In [ ]:
!rm -f output/*

In [ ]:
cfg.merge_from_file('configs/example_resnet152_margin.yaml')
train(cfg)

In [ ]:
!mkdir drive/MyDrive/resnet152
!cp output/model_002800.pth drive/MyDrive/resnet152

In [ ]:
!rm -f output/*